# Test des données récupérées par API

Ce fichier prend en compte les requêtes pour l'appel API <br><br>
Objet du traitement : récupération des salaires bruts et des effectifs dans le secteur de la restauration en prennant en compte la correction des valeurs 
saisonnières (CVS)

À faire avec streamlit :<br>
- Un titre principal
- Une colonne comprenant un menu déroulant (hébergement / restauration) et une table des données
- Une colonne avec un [diagramme linéaire](https://docs.streamlit.io/library/api-reference/charts/st.line_chart)

Date : 08-01-2024 <br>
Éditeur : <b>Laurent REYNAUD</b>

In [1]:
import requests
import json
import polars as pl

### URL API :

In [3]:
url_api = "https://open.urssaf.fr/explore/dataset/effectifs-salaries-et-masse-salariale-du-secteur-prive-france-entiere-x-na88/api/?refine.grand_secteur_d_activite=GS4+H%C3%B4tellerie-restauration&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IlNVTSIsInlBeGlzIjoiZWZmZWN0aWZzX3NhbGFyaWVzX2N2cyIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiM2NmMyYTUifSx7ImFsaWduTW9udGgiOnRydWUsInR5cGUiOiJsaW5lIiwiZnVuYyI6IlNVTSIsInlBeGlzIjoibWFzc2Vfc2FsYXJpYWxlX2N2cyIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiNmYzhkNjIifV0sInhBeGlzIjoiZGVybmllcl9qb3VyX2R1X3RyaW1lc3RyZSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6ImRheSIsInNvcnQiOiIiLCJjb25maWciOnsiZGF0YXNldCI6ImVmZmVjdGlmcy1zYWxhcmllcy1ldC1tYXNzZS1zYWxhcmlhbGUtZHUtc2VjdGV1ci1wcml2ZS1mcmFuY2UtZW50aWVyZS14LW5hODgiLCJvcHRpb25zIjp7InJlZmluZS5ncmFuZF9zZWN0ZXVyX2RfYWN0aXZpdGUiOiJHUzQgSFx1MDBGNHRlbGxlcmllLXJlc3RhdXJhdGlvbiJ9fX1dLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D"

### Requête SQL avec SELECT :

annee, trimestre, effectifs_salaries_cvs, masse_salariale_cvs

### Requête SQL avec WHERE :

secteur_na88i="56 Restauration" and annee>date'2012'

### Requête SQL avec GROUP_BY :

### Requête SQL avec ORDER BY :

annee, trimestre

### Requête SQL avec LIMIT :

100

### Variables à saisir avec les requêtes SQL WHERE / LIMIT... ci-avant :

In [8]:
sector = input("Hébergement ou Restauration ?")
if sector == "Hebergement":
    selected_sector = '55 Hébergement'
else:
    selected_sector = '56 Restauration'


### URL

[Encodage-pourcent](https://fr.wikipedia.org/wiki/Encodage-pourcent)

In [9]:
# Accès à l'URL avec le requêtage appliqué : 
# url = f"https://open.urssaf.fr/api/explore/v2.1/catalog/datasets/effectifs-salaries-et-masse-salariale-du-secteur-prive-france-entiere-x-na88/records?select=annee%2C%20trimestre%2C%20effectifs_salaries_cvs%2C%20masse_salariale_cvs&where=secteur_na88i%3D%2256%20Restauration%22%20and%20annee%3Edate%272012%27&order_by=annee%2C%20trimestre&limit=100&refine=grand_secteur_d_activite%3A%22GS4%20H%C3%B4tellerie-restauration%22"
url = f"https://open.urssaf.fr/api/explore/v2.1/catalog/datasets/effectifs-salaries-et-masse-salariale-du-secteur-prive-france-entiere-x-na88/records?select=annee%2C%20trimestre%2C%20effectifs_salaries_cvs%2C%20masse_salariale_cvs&where=secteur_na88i%3D%22{selected_sector}%22%20and%20annee%3Edate%272012%27&order_by=annee%2C%20trimestre&limit=100&refine=grand_secteur_d_activite%3A%22GS4%20H%C3%B4tellerie-restauration%22"

In [10]:
# Requête url
response = requests.get(url)

In [11]:
# Code statut html
response.status_code

200

In [12]:
# Récupération du contenu au format json
data = json.loads(response.content)

In [22]:
# Récupération des données converties en DF polars
df = pl.DataFrame(data['results'])

In [21]:
# Moyenne des données par année civile
df = (df.groupby('annee')
      .agg(pl.mean(['effectifs_salaries_cvs', 'masse_salariale_cvs'])
           .cast(pl.Int32))
      .sort(by='annee'))
df

C:\Users\LRCOM\AppData\Local\Temp\ipykernel_15464\2255914483.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  df = (df.groupby('annee')


annee,effectifs_salaries_cvs,masse_salariale_cvs
str,i32,i32
"""2013""",247612,1392473674
"""2014""",246791,1411512483
"""2015""",248023,1442101383
"""2016""",251857,1471837710
"""2017""",252874,1522014547
"""2018""",258559,1590974875
"""2019""",262513,1655328748
"""2020""",230783,1101666695
"""2021""",242198,1260706112
